In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
import umap
from sklearn.impute import KNNImputer
from sklearn.linear_model import LassoCV
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.utils import shuffle
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Times New Roman'
import time
import platform
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from keras import regularizers
import io
import re
seed=2023

In [41]:
pd.set_option('display.max_columns',100)

In [42]:
def add_space_around_punctuation(texts):

    texts = [text.lower() for text in texts]

    punctuation_chars = r'!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    pattern_before = re.compile(r'(?<!\s)([' + re.escape(punctuation_chars) + r'])')
    pattern_after = re.compile(r'([' + re.escape(punctuation_chars) + r'])(?!\s|$)')
    
    texts_with_spaces = [pattern_before.sub(r' \1', text) for text in texts]
    texts_with_spaces = [pattern_after.sub(r'\1 ', text) for text in texts_with_spaces]

    texts_with_single_spaces = [re.sub(r'\s+', ' ', text) for text in texts_with_spaces]

    return texts_with_single_spaces

In [43]:
df = pd.read_csv('E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim1_2_merged1_baidu.csv')
df

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [44]:
df0 = df.copy()
temp1 = df0['long_title'].values
temp2 = df0['icd_title'].values
temp3 = []
for i in range(df0.shape[0]):
    if pd.isna(temp1[i]) & pd.isna(temp2[i]):
        temp3.append(np.nan)
    elif pd.isna(temp1[i]):
        temp3.append(temp2[i])
    elif pd.isna(temp2[i]):
        temp3.append(temp1[i])
    elif temp1[i].lower() in temp2[i].lower():
        temp3.append(temp2[i])
    elif temp2[i].lower() in temp1[i].lower():
        temp3.append(temp1[i])
    else:
        temp3.append(temp1[i]+', '+temp2[i])
df0['icd_title'] = temp3
df0 = df0.drop(['long_title'], axis=1)
df0

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~Absolute Neutrophil Count~52075~K/uL_,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN

In [45]:
temp1 = df0['chiefcomplaint'].values
temp2 = df0['Chief Complaint'].values
temp3 = []
for i in range(df0.shape[0]):
    if pd.isna(temp1[i]) & pd.isna(temp2[i]):
        temp3.append(np.nan)
    elif pd.isna(temp1[i]):
        temp3.append(temp2[i])
    elif pd.isna(temp2[i]):
        temp3.append(temp1[i])
    elif temp1[i].lower() in temp2[i].lower():
        temp3.append(temp2[i])
    elif temp2[i].lower() in temp1[i].lower():
        temp3.append(temp1[i])
    else:
        temp3.append(temp1[i]+', '+temp2[i])
df0['ChiefComplaint_combined'] = temp3
df0 = df0.drop(['chiefcomplaint', 'Chief Complaint'], axis=1)
df0.rename(columns={'ChiefComplaint_combined':'Chief Complaint'},inplace=True)
df0

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~Absolute Monocyte Count~52074~K/uL_,Hematology~Blood~Absolute Neutrophil Count~52075~K/uL_,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,History of Present Illness,Past Medical History,Family History,hospital_expire_flag,Chief Complaint
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Extraction of immune cell-related features

In [47]:
features = pd.read_csv("E:/CNSI_project/demo_and_data_3.0/Raw_data/mimic-iv-2.2/hosp/labitems_filtered1_baidu.csv")
features

,itemid,label,fluid,category,label_ch,valueuom
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas,肺泡动脉梯度,mm Hg
1,50802,Base Excess,Blood,Blood Gas,基本超额,mEq/L
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,计算的碳酸氢盐，全血,mEq/L
3,50804,Calculated Total CO2,Blood,Blood Gas,计算的总CO2,mEq/L
4,50805,Carboxyhemoglobin,Blood,Blood Gas,羧基血红蛋白,%
...,...,...,...,...,...,...
901,53150,Anti Hbs,Blood,Chemistry,抗Hb,NaN
902,53151,Anti-la,Blood,Chemistry,Anti-la,NaN
903,53152,HIV FINAL,Blood,Chemistry,HIV最终结果,NaN
904,53153,HIV Screen,Blood,Chemistry,HIV筛查,NaN


In [48]:
features.fluid.value_counts()

Blood                  807
Cerebrospinal Fluid     99
Name: fluid, dtype: int64

In [49]:
# Pick out data with Cerebrospinal Fluid in the column name
df_csf = df0.filter(like="Cerebrospinal Fluid", axis=1)
df_csf

,Chemistry~Cerebrospinal Fluid~<Albumin>~51776~mg/dL,"Chemistry~Cerebrospinal Fluid~Bilirubin, Total, CSF~51783~mg/dL","Chemistry~Cerebrospinal Fluid~Glucose, CSF~51790~mg/dL",Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load~51791~log10 copies/mL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%
0,NaN,NaN,59.0,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,5.0,0.0,NaN
1,NaN,NaN,55.0,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0,NaN
2,NaN,NaN,61.0,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,2.0,8.0,NaN,32.0,NaN,NaN,NaN,NaN,17.0,NaN,334.0,4.0,NaN
3,NaN,NaN,61.0,NaN,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,49.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10.0,NaN,430.0,10.0,NaN
4,NaN,NaN,60.0,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,0.0,NaN,510.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,NaN,NaN,1.0,NaN,57.0,590.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,90.0,NaN,250.0,7800.0,NaN
11855,NaN,NaN,42.0,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,8.0,NaN,1.0,3.0,NaN
11856,NaN,NaN,87.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,9.0,NaN,4.0,0.0,NaN
11857,NaN,NaN,57.0,NaN,23.0,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,NaN,NaN,NaN,8.0,NaN,NaN,14.0,NaN,1.0,NaN,4.0,26.0,NaN


In [50]:
df_csf.columns.tolist()

['Chemistry~Cerebrospinal Fluid~<Albumin>~51776~mg/dL',
 'Chemistry~Cerebrospinal Fluid~Bilirubin, Total, CSF~51783~mg/dL',
 'Chemistry~Cerebrospinal Fluid~Glucose, CSF~51790~mg/dL',
 'Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load~51791~log10 copies/mL',
 'Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L',
 'Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL',
 'Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%',
 'Hematology~Cerebrospinal Fluid~Bands~52223~%',
 'Hematology~Cerebrospinal Fluid~Basophils~52225~%',
 'Hematology~Cerebrospinal Fluid~Blasts~52227~%',
 'Hematology~Cerebrospinal Fluid~Eosinophils~52256~%',
 'Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%',
 'Hematology~Cerebrospinal Fluid~Lining Cell~52263~%',
 'Hematology~Cerebrospinal Fluid~Lymphs~52264~%',
 'Hematology~Cerebrospinal Fluid~Macrophage~52266~%',
 'Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%',
 'Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%',

In [59]:
column_names_detailed = df_csf.columns.tolist()

data_detailed = {
    "En": column_names_detailed,
    "Ch": [
        "脑脊液白蛋白",
        "脑脊液总胆红素",
        "脑脊液葡萄糖",
        "脑脊液 HIV 1 病毒载量",
        "脑脊液乳酸脱氢酶",
        "脑脊液总蛋白",
        "脑脊液异常淋巴细胞",
        "脑脊液杆状核细胞",
        "脑脊液嗜碱性粒细胞",
        "脑脊液原始细胞",
        "脑脊液嗜酸性粒细胞",
        "脑脊液红细胞压积",
        "脑脊液被覆细胞",
        "脑脊液淋巴细胞",
        "脑脊液巨噬细胞",
        "脑脊液间皮细胞",
        "脑脊液后骨髓细胞",
        "脑脊液单核细胞",
        "脑脊液骨髓细胞",
        "脑脊液有核红细胞",
        "脑脊液其他细胞",
        "脑脊液浆细胞",
        "脑脊液中性粒细胞",
        "脑脊液前骨髓细胞",
        "脑脊液红细胞",
        "脑脊液总有核细胞",
        "脑脊液幼稚细胞"
    ]
}

df_translation_detailed = pd.DataFrame(data_detailed)
df_translation_detailed

,En,Ch
0,Chemistry~Cerebrospinal Fluid~<Albumin>~51776~...,脑脊液白蛋白
1,"Chemistry~Cerebrospinal Fluid~Bilirubin, Total...",脑脊液总胆红素
2,"Chemistry~Cerebrospinal Fluid~Glucose, CSF~517...",脑脊液葡萄糖
3,Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load...,脑脊液 HIV 1 病毒载量
4,Chemistry~Cerebrospinal Fluid~Lactate Dehydrog...,脑脊液乳酸脱氢酶
5,"Chemistry~Cerebrospinal Fluid~Total Protein, C...",脑脊液总蛋白
6,Hematology~Cerebrospinal Fluid~Atypical Lympho...,脑脊液异常淋巴细胞
7,Hematology~Cerebrospinal Fluid~Bands~52223~%,脑脊液杆状核细胞
8,Hematology~Cerebrospinal Fluid~Basophils~52225~%,脑脊液嗜碱性粒细胞
9,Hematology~Cerebrospinal Fluid~Blasts~52227~%,脑脊液原始细胞


In [52]:
aim1_withID = pd.read_csv("E:/CNSI_project/demo_and_data_5.0/Processed_data/paper/aim1_datasetWithID.csv")
aim1_withID

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Ploys,resprate,pain,acuity,culture
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0


In [55]:
IDaim1 = aim1_withID[['subject_id','hadm_id','chartdate']].drop_duplicates(keep='first')
IDaim1

,subject_id,hadm_id,chartdate
0,10002557,NaN,2147-03-26
1,10002559,NaN,2179-06-05
2,10003019,21616816.0,2174-09-18
3,10005808,26543907.0,2119-01-02
4,10006269,27357430.0,2124-06-26
...,...,...,...
9256,19994772,29606061.0,2180-12-18
9257,19994772,NaN,2181-03-13
9258,19995080,23057109.0,2147-07-13
9259,19997843,NaN,2120-11-18


In [63]:
df1 = IDaim1.merge(df0, on=['subject_id','hadm_id','chartdate'], how='left')
df2 = df1[['subject_id','hadm_id','chartdate']+df_csf.columns.tolist()+['culture']]
df2

,subject_id,hadm_id,chartdate,Chemistry~Cerebrospinal Fluid~<Albumin>~51776~mg/dL,"Chemistry~Cerebrospinal Fluid~Bilirubin, Total, CSF~51783~mg/dL","Chemistry~Cerebrospinal Fluid~Glucose, CSF~51790~mg/dL",Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load~51791~log10 copies/mL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,culture
0,10002557,NaN,2147-03-26,NaN,NaN,55.0,NaN,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,2.0,1.0,NaN,0
1,10002559,NaN,2179-06-05,NaN,NaN,61.0,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,2.0,8.0,NaN,32.0,NaN,NaN,NaN,NaN,17.0,NaN,334.0,4.0,NaN,0
2,10003019,21616816.0,2174-09-18,NaN,NaN,61.0,NaN,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,49.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10.0,NaN,430.0,10.0,NaN,0
3,10005808,26543907.0,2119-01-02,NaN,NaN,60.0,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,0.0,NaN,510.0,1.0,NaN,0
4,10006269,27357430.0,2124-06-26,NaN,NaN,57.0,NaN,NaN,100.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,96.0,NaN,NaN,NaN,3.0,NaN,NaN,0.0,NaN,1.0,NaN,2.0,141.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,70.0,NaN,NaN,295.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,100.0,NaN,5100.0,18600.0,NaN,0
9257,19994772,NaN,2181-03-13,NaN,NaN,1.0,NaN,57.0,590.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,90.0,NaN,250.0,7800.0,NaN,0
9258,19995080,23057109.0,2147-07-13,NaN,NaN,42.0,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,8.0,NaN,1.0,3.0,NaN,0
9259,19997843,NaN,2120-11-18,NaN,NaN,87.0,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,9.0,NaN,4.0,0.0,NaN,0


In [64]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9261 entries, 0 to 9260
Data columns (total 31 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   subject_id                                                            9261 non-null   int64  
 1   hadm_id                                                               6101 non-null   float64
 2   chartdate                                                             9261 non-null   object 
 3   Chemistry~Cerebrospinal Fluid~<Albumin>~51776~mg/dL                   25 non-null     float64
 4   Chemistry~Cerebrospinal Fluid~Bilirubin, Total, CSF~51783~mg/dL       3 non-null      float64
 5   Chemistry~Cerebrospinal Fluid~Glucose, CSF~51790~mg/dL                8860 non-null   float64
 6   Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load~51791~log10 copies/mL  3 non-null      float64
 7

In [65]:
df2[df2['culture']==0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9016 entries, 0 to 9260
Data columns (total 31 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   subject_id                                                            9016 non-null   int64  
 1   hadm_id                                                               5897 non-null   float64
 2   chartdate                                                             9016 non-null   object 
 3   Chemistry~Cerebrospinal Fluid~<Albumin>~51776~mg/dL                   25 non-null     float64
 4   Chemistry~Cerebrospinal Fluid~Bilirubin, Total, CSF~51783~mg/dL       3 non-null      float64
 5   Chemistry~Cerebrospinal Fluid~Glucose, CSF~51790~mg/dL                8642 non-null   float64
 6   Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load~51791~log10 copies/mL  3 non-null      float64
 7

In [94]:
non_null_counts = df2.count()
non_null_df = pd.DataFrame({
    "En": non_null_counts.index,
    "non_null_counts": non_null_counts.values
})
non_null_df

,En,non_null_counts
0,subject_id,9261
1,hadm_id,6101
2,chartdate,9261
3,Chemistry~Cerebrospinal Fluid~<Albumin>~51776~...,25
4,"Chemistry~Cerebrospinal Fluid~Bilirubin, Total...",3
5,"Chemistry~Cerebrospinal Fluid~Glucose, CSF~517...",8860
6,Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load...,3
7,Chemistry~Cerebrospinal Fluid~Lactate Dehydrog...,1350
8,"Chemistry~Cerebrospinal Fluid~Total Protein, C...",8975
9,Hematology~Cerebrospinal Fluid~Atypical Lympho...,479


In [96]:
csf_ch_count = df_translation_detailed.merge(non_null_df, on='En', how='left')
csf_ch_count

,En,Ch,non_null_counts
0,Chemistry~Cerebrospinal Fluid~<Albumin>~51776~...,脑脊液白蛋白,25
1,"Chemistry~Cerebrospinal Fluid~Bilirubin, Total...",脑脊液总胆红素,3
2,"Chemistry~Cerebrospinal Fluid~Glucose, CSF~517...",脑脊液葡萄糖,8860
3,Chemistry~Cerebrospinal Fluid~HIV 1 Viral Load...,脑脊液 HIV 1 病毒载量,3
4,Chemistry~Cerebrospinal Fluid~Lactate Dehydrog...,脑脊液乳酸脱氢酶,1350
5,"Chemistry~Cerebrospinal Fluid~Total Protein, C...",脑脊液总蛋白,8975
6,Hematology~Cerebrospinal Fluid~Atypical Lympho...,脑脊液异常淋巴细胞,479
7,Hematology~Cerebrospinal Fluid~Bands~52223~%,脑脊液杆状核细胞,316
8,Hematology~Cerebrospinal Fluid~Basophils~52225~%,脑脊液嗜碱性粒细胞,189
9,Hematology~Cerebrospinal Fluid~Blasts~52227~%,脑脊液原始细胞,16


In [99]:
# bands are immature neutrophils that can be placed in leukocytes and not counted as neutrophils
# Leukocytes include: (Basophils, Eosinophils, Polys), (Lymphs, Monocytes, Macrophage, Plasma), Atypical Lymphocytes, Bands
i = [6,7,8,10,13,14,17,21,22,25]
features = csf_ch_count.iloc[i]['En'].tolist()
features

['Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%',
 'Hematology~Cerebrospinal Fluid~Bands~52223~%',
 'Hematology~Cerebrospinal Fluid~Basophils~52225~%',
 'Hematology~Cerebrospinal Fluid~Eosinophils~52256~%',
 'Hematology~Cerebrospinal Fluid~Lymphs~52264~%',
 'Hematology~Cerebrospinal Fluid~Macrophage~52266~%',
 'Hematology~Cerebrospinal Fluid~Monocytes~52272~%',
 'Hematology~Cerebrospinal Fluid~Plasma~52279~%',
 'Hematology~Cerebrospinal Fluid~Polys~52281~%',
 'Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL']

In [104]:
# Merge the selected features into the existing aim1 dataset
dff0 = df0[['subject_id','hadm_id','chartdate']+features]
dff0

,subject_id,hadm_id,chartdate,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL"
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,100.0,NaN,0.0,NaN,0.0,0.0
1,10002557,NaN,2147-03-26,NaN,NaN,NaN,NaN,100.0,NaN,0.0,NaN,0.0,1.0
2,10002559,NaN,2179-06-05,NaN,NaN,NaN,NaN,43.0,2.0,32.0,NaN,17.0,4.0
3,10003019,21616816.0,2174-09-18,NaN,NaN,NaN,NaN,41.0,49.0,0.0,NaN,10.0,10.0
4,10005808,26543907.0,2119-01-02,NaN,NaN,NaN,NaN,62.0,NaN,38.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,19994772,NaN,2181-03-13,NaN,NaN,1.0,NaN,1.0,NaN,8.0,NaN,90.0,7800.0
11855,19995080,23057109.0,2147-07-13,NaN,NaN,NaN,NaN,88.0,NaN,4.0,NaN,8.0,3.0
11856,19997843,NaN,2120-11-18,NaN,NaN,NaN,NaN,55.0,NaN,36.0,NaN,9.0,0.0
11857,19999784,24935234.0,2119-07-07,NaN,NaN,NaN,NaN,77.0,NaN,8.0,NaN,1.0,26.0


In [123]:
# Can see that all the FEATURES are percentages except Total Nucleated Cells, and this record is fine in the MIMIC system.
dff0.describe()

,subject_id,hadm_id,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL"
count,1.185900e+04,7.093000e+03,598.000000,357.000000,225.000000,1105.000000,11599.000000,2408.000000,11599.000000,129.000000,11600.000000,11600.000000
mean,1.502213e+07,2.498387e+07,2.867893,3.389356,1.595556,3.089593,60.242780,15.310631,16.456764,3.945736,14.930431,121.214181
std,2.884246e+06,2.925671e+06,5.425692,4.422516,2.442367,6.977586,30.747536,18.222528,17.421932,12.375196,26.313513,2230.748877
min,1.000122e+07,2.000126e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.255134e+07,2.242626e+07,1.000000,1.000000,1.000000,1.000000,38.000000,3.000000,3.000000,1.000000,0.000000,1.000000
50%,1.504778e+07,2.501417e+07,2.000000,2.000000,1.000000,1.000000,69.000000,8.000000,11.000000,1.000000,1.000000,2.000000
75%,1.746643e+07,2.758815e+07,3.000000,4.000000,2.000000,3.000000,86.000000,20.000000,25.000000,3.000000,15.000000,7.000000
max,1.999978e+07,2.999910e+07,100.000000,28.000000,25.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,215373.000000


In [105]:
aim1_withID_corrected = aim1_withID.merge(dff0, on=['subject_id','hadm_id','chartdate'], how='left')
aim1_withID_corrected

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Ploys,resprate,pain,acuity,culture,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL"
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,100.0,NaN,0.0,NaN,0.0,1.0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,43.0,2.0,32.0,NaN,17.0,4.0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,NaN,NaN,NaN,NaN,41.0,49.0,0.0,NaN,10.0,10.0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,62.0,NaN,38.0,NaN,0.0,1.0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,NaN,NaN,1.0,NaN,96.0,NaN,3.0,NaN,1.0,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,100.0,18600.0
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,1.0,NaN,8.0,NaN,90.0,7800.0
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,88.0,NaN,4.0,NaN,8.0,3.0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,55.0,NaN,36.0,NaN,9.0,0.0


In [107]:
# Remove duplicate features
aim1_withID_corrected1 = aim1_withID_corrected.drop(columns=['Hematology~Cerebrospinal Fluid~Lymphs~52264~%',
                                   'Hematology~Cerebrospinal Fluid~Macrophage~52266~%',
                                   'Hematology~Cerebrospinal Fluid~Polys~52281~%'])
aim1_withID_corrected1

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Ploys,resprate,pain,acuity,culture,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,"Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL"
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,1.0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,32.0,NaN,4.0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,NaN,NaN,NaN,NaN,0.0,NaN,10.0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,38.0,NaN,1.0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,NaN,NaN,1.0,NaN,3.0,NaN,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,18600.0
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,8.0,NaN,7800.0
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,4.0,NaN,3.0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,36.0,NaN,0.0


In [137]:
# Modify column names
aim1_withID_corrected2 = aim1_withID_corrected1.copy()
new_column_names = {
    "Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%" : "C_AtypicalLymphocytes",
    "Hematology~Cerebrospinal Fluid~Bands~52223~%" : "C_Bands",
    "Hematology~Cerebrospinal Fluid~Basophils~52225~%" : "C_Basophils",
    "Hematology~Cerebrospinal Fluid~Eosinophils~52256~%" : "C_Eosinophils",
    "Hematology~Cerebrospinal Fluid~Monocytes~52272~%" : "C_Monocytes",
    "Hematology~Cerebrospinal Fluid~Plasma~52279~%" : "C_Plasma",
    "Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL" : "C_TotalNucleatedCells",
    
    "C_Ploys" : "C_Polys"
                   }
aim1_withID_corrected2 = aim1_withID_corrected2.rename(columns=new_column_names)
aim1_withID_corrected2

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_AtypicalLymphocytes,C_Bands,C_Basophils,C_Eosinophils,C_Monocytes,C_Plasma,C_TotalNucleatedCells
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,1.0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,32.0,NaN,4.0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,NaN,NaN,NaN,NaN,0.0,NaN,10.0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,38.0,NaN,1.0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,NaN,NaN,1.0,NaN,3.0,NaN,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,18600.0
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,8.0,NaN,7800.0
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,4.0,NaN,3.0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,36.0,NaN,0.0


In [138]:
# Count these white blood cells（(Basophils,Eosinophils,Polys), (Lymphs,Monocytes,Macrophage,Plasma), Atypical Lymphocytes, Bands）
aim1_withID_corrected3 = aim1_withID_corrected2.copy()
aim1_withID_corrected3["C_Basophils_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Basophils"]/100
aim1_withID_corrected3["C_Eosinophils_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Eosinophils"]/100
aim1_withID_corrected3["C_Polys_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Polys"]/100
aim1_withID_corrected3["C_Lymphs_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Lymphs"]/100
aim1_withID_corrected3["C_Monocytes_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Monocytes"]/100
aim1_withID_corrected3["C_Macrophage_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Macrophage"]/100
aim1_withID_corrected3["C_Plasma_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Plasma"]/100
aim1_withID_corrected3["C_AtypicalLymphocytes_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_AtypicalLymphocytes"]/100
aim1_withID_corrected3["C_Bands_c"] = aim1_withID_corrected3["C_TotalNucleatedCells"]*aim1_withID_corrected3["C_Bands"]/100
aim1_withID_corrected3

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_AtypicalLymphocytes,C_Bands,C_Basophils,C_Eosinophils,C_Monocytes,C_Plasma,C_TotalNucleatedCells,C_Basophils_c,C_Eosinophils_c,C_Polys_c,C_Lymphs_c,C_Monocytes_c,C_Macrophage_c,C_Plasma_c,C_AtypicalLymphocytes_c,C_Bands_c
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,0.00,1.00,0.00,NaN,NaN,NaN,NaN
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,32.0,NaN,4.0,NaN,NaN,0.68,1.72,1.28,0.08,NaN,NaN,NaN
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,NaN,NaN,NaN,NaN,0.0,NaN,10.0,NaN,NaN,1.00,4.10,0.00,4.90,NaN,NaN,NaN
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,38.0,NaN,1.0,NaN,NaN,0.00,0.62,0.38,NaN,NaN,NaN,NaN
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,NaN,NaN,1.0,NaN,3.0,NaN,141.0,1.41,NaN,1.41,135.36,4.23,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,18600.0,NaN,NaN,18600.00,0.00,0.00,NaN,NaN,NaN,NaN
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,8.0,NaN,7800.0,78.00,NaN,7020.00,78.00,624.00,NaN,NaN,NaN,NaN
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,0.24,2.64,0.12,NaN,NaN,NaN,NaN
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,36.0,NaN,0.0,NaN,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN


In [152]:
# Totalling the number of white blood cells
aim1_withID_corrected4 = aim1_withID_corrected3.copy()
aim1_withID_corrected4["C_LeukocyteCount"] = aim1_withID_corrected4[
    [
        'C_Basophils_c',
        'C_Eosinophils_c',
        'C_Polys_c',
        'C_Lymphs_c',
        'C_Monocytes_c',
        'C_Macrophage_c',
        'C_Plasma_c',
        'C_AtypicalLymphocytes_c',
        'C_Bands_c'
    ]
].sum(axis=1, min_count=1)

# sum(axis=1, min_count=1) will sum along the rows, if all 9 columns have NaN values, the result is NaN.
If all 9 columns are NaN, the result will be NaN. # min_count=1 means at least one non-NaN value can be added, otherwise the result will be NaN.
# i.e. if all 9 columns are NaN, the result will be NaN.
aim1_withID_corrected4

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_AtypicalLymphocytes,C_Bands,C_Basophils,C_Eosinophils,C_Monocytes,C_Plasma,C_TotalNucleatedCells,C_Basophils_c,C_Eosinophils_c,C_Polys_c,C_Lymphs_c,C_Monocytes_c,C_Macrophage_c,C_Plasma_c,C_AtypicalLymphocytes_c,C_Bands_c,C_LeukocyteCount
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,0.00,1.00,0.00,NaN,NaN,NaN,NaN,1.00
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,32.0,NaN,4.0,NaN,NaN,0.68,1.72,1.28,0.08,NaN,NaN,NaN,3.76
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,NaN,NaN,NaN,NaN,0.0,NaN,10.0,NaN,NaN,1.00,4.10,0.00,4.90,NaN,NaN,NaN,10.00
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,38.0,NaN,1.0,NaN,NaN,0.00,0.62,0.38,NaN,NaN,NaN,NaN,1.00
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,NaN,NaN,1.0,NaN,3.0,NaN,141.0,1.41,NaN,1.41,135.36,4.23,NaN,NaN,NaN,NaN,142.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,18600.0,NaN,NaN,18600.00,0.00,0.00,NaN,NaN,NaN,NaN,18600.00
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,8.0,NaN,7800.0,78.00,NaN,7020.00,78.00,624.00,NaN,NaN,NaN,NaN,7800.00
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,0.24,2.64,0.12,NaN,NaN,NaN,NaN,3.00
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,36.0,NaN,0.0,NaN,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN,0.00


In [153]:
col_drop = aim1_withID_corrected4.columns.tolist()[30:-1]
col_drop

['C_AtypicalLymphocytes',
 'C_Bands',
 'C_Basophils',
 'C_Eosinophils',
 'C_Monocytes',
 'C_Plasma',
 'C_TotalNucleatedCells',
 'C_Basophils_c',
 'C_Eosinophils_c',
 'C_Polys_c',
 'C_Lymphs_c',
 'C_Monocytes_c',
 'C_Macrophage_c',
 'C_Plasma_c',
 'C_AtypicalLymphocytes_c',
 'C_Bands_c']

In [154]:
# Discard the redundant columns
aim1_withID_corrected5 = aim1_withID_corrected4.copy()
aim1_withID_corrected5 = aim1_withID_corrected5.drop(columns=col_drop)
aim1_withID_corrected5

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_LeukocyteCount
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,1.00
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,3.76
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,10.00
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,1.00
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,142.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,18600.00
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,7800.00
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,3.00
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,0.00


In [193]:
aim1_withID_corrected6 = aim1_withID_corrected5.copy()
aim1_withID_corrected6['label_from_paper'] = aim1_withID_corrected6['culture']
aim1_withID_corrected6.loc[(aim1_withID_corrected6["culture"]==0) & (aim1_withID_corrected6["C_LeukocyteCount"]>1000) & (aim1_withID_corrected6["C_Polys"]>50), "label_from_paper"] = 1
aim1_withID_corrected6

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_LeukocyteCount,label_from_paper
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,3.76,0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,10.00,0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,142.41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,18600.00,1
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,7800.00,1
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,3.00,0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,0.00,0


In [194]:
aim1_withID_corrected6['culture'].value_counts()

0    9016
1     245
Name: culture, dtype: int64

In [195]:
# With a Leukocyte of 250, there are 325 zeros that become 1s
# With a Leukocyte of 1000, 132 zeros become 1s
aim1_withID_corrected6['label_from_paper'].value_counts()

0    8884
1     377
Name: label_from_paper, dtype: int64

In [196]:
# Combining information from MIMIC's ICDs
aim1_correctedFromICDtext = pd.read_excel("E:/CNSI_project/demo_and_data_5.0/Processed_data/paper/aim_1_2_mergedFiltered_correctedLabel.xlsx")
aim1_correctedFromICDtext = aim1_correctedFromICDtext[aim1_correctedFromICDtext['culture'].notna()]
aim1_correctedFromICDtext

,subject_id,hadm_id,chartdate,C_Lymphs,C_Macrophage,C_Ploys,C_Eosinophils,C_Polys,icd_title,Corrected culture result label,culture
0,10527074,27892723.0,2171-02-14,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
1,10570948,24067781.0,2158-03-01,35.0,9.0,0.0,NaN,NaN,NaN,1,1.0
2,11929508,NaN,2136-05-09,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
3,12687602,NaN,2188-08-30,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
4,13148393,NaN,2141-10-03,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9256,19834949,27411859.0,2179-11-06,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
9257,19957285,20267759.0,2118-09-22,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
9258,19957285,20267759.0,2118-09-24,NaN,NaN,NaN,NaN,NaN,"Alkalosis; Anxiety disorder, unspecified; Bact...",0,0.0
9259,19982539,23136520.0,2175-05-30,NaN,NaN,NaN,NaN,NaN,Candidal stomatitis; Cerebral edema; Cerebral ...,0,0.0


In [197]:
aim1_correctedFromICDtext['culture'].value_counts()

0.0    9016
1.0     245
Name: culture, dtype: int64

In [198]:
aim1_correctedFromICDtext['Corrected culture result label'].value_counts()

0    8985
1     243
2      33
Name: Corrected culture result label, dtype: int64

In [199]:
# Change label 2 to 1
aim1_correctedFromICDtext1 = aim1_correctedFromICDtext.copy()
aim1_correctedFromICDtext1.loc[aim1_correctedFromICDtext1['Corrected culture result label']==2, "Corrected culture result label"] = 1
aim1_correctedFromICDtext1

,subject_id,hadm_id,chartdate,C_Lymphs,C_Macrophage,C_Ploys,C_Eosinophils,C_Polys,icd_title,Corrected culture result label,culture
0,10527074,27892723.0,2171-02-14,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
1,10570948,24067781.0,2158-03-01,35.0,9.0,0.0,NaN,NaN,NaN,1,1.0
2,11929508,NaN,2136-05-09,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
3,12687602,NaN,2188-08-30,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
4,13148393,NaN,2141-10-03,0.0,NaN,100.0,NaN,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9256,19834949,27411859.0,2179-11-06,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
9257,19957285,20267759.0,2118-09-22,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
9258,19957285,20267759.0,2118-09-24,NaN,NaN,NaN,NaN,NaN,"Alkalosis; Anxiety disorder, unspecified; Bact...",0,0.0
9259,19982539,23136520.0,2175-05-30,NaN,NaN,NaN,NaN,NaN,Candidal stomatitis; Cerebral edema; Cerebral ...,0,0.0


In [200]:
aim1_correctedFromICDtext1['Corrected culture result label'].value_counts()

0    8985
1     276
Name: Corrected culture result label, dtype: int64

In [201]:
# Convert date types to string types
aim1_correctedFromICDtext2 = aim1_correctedFromICDtext1.copy()
aim1_correctedFromICDtext2['chartdate'] = aim1_correctedFromICDtext2['chartdate'].astype(str)

In [202]:
# merge
aim1_withID_corrected7 = aim1_withID_corrected6.copy()
aim1_withID_corrected7 = aim1_withID_corrected7.merge(aim1_correctedFromICDtext2[['subject_id','hadm_id','chartdate','Corrected culture result label']], 
                                                      on=['subject_id','hadm_id','chartdate'], how='left')
aim1_withID_corrected7

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_LeukocyteCount,label_from_paper,Corrected culture result label
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0,0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,3.76,0,0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,10.00,0,0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0,0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,142.41,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,18600.00,1,0
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,7800.00,1,0
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,3.00,0,0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,0.00,0,0


# The three labels of aim1 are merged into one.

In [203]:
aim1_dataset_label_3to1 = aim1_withID_corrected7.copy()
aim1_dataset_label_3to1['Label_merged'] = 0
aim1_dataset_label_3to1.loc[(aim1_dataset_label_3to1['culture']==1) | (aim1_dataset_label_3to1['label_from_paper']==1) | (aim1_dataset_label_3to1['Corrected culture result label']==1), 'Label_merged'] = 1
aim1_dataset_label_3to1

,subject_id,hadm_id,chartdate,B_pO2,B_Albumin,B_Creatinine,B_Glucose,B_H,B_I,B_LactateDehydrogenase,B_Potassium,B_Sodium,B_Hemoglobin,B_lymphocytes,B_MCHC,B_MonocytesPercentage,B_Platelet,B_PTT,C_Glucose,C_Protein,B_MonocyteCount,B_ImmatureGranulocytes,B_RDW-SD,C_Lymphs,C_Macrophage,C_Polys,resprate,pain,acuity,culture,C_LeukocyteCount,label_from_paper,Corrected culture result label,Label_merged
0,10002557,NaN,2147-03-26,NaN,NaN,0.7,92.0,NaN,NaN,NaN,3.7,144.0,11.7,21.9,33.0,3.8,226.0,27.5,55.0,29.0,NaN,NaN,NaN,100.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0,0,0
1,10002559,NaN,2179-06-05,NaN,3.6,0.6,95.0,NaN,NaN,187.0,3.7,137.0,12.6,11.2,34.3,5.9,198.0,NaN,61.0,24.0,NaN,NaN,NaN,43.0,2.0,17.0,NaN,NaN,NaN,0,3.76,0,0,0
2,10003019,21616816.0,2174-09-18,NaN,2.6,1.2,76.0,NaN,NaN,NaN,4.9,137.0,9.7,NaN,30.8,NaN,557.0,NaN,61.0,75.0,NaN,NaN,NaN,41.0,49.0,10.0,16.0,13.0,2.0,0,10.00,0,0,0
3,10005808,26543907.0,2119-01-02,NaN,NaN,0.8,105.0,NaN,NaN,NaN,3.9,142.0,15.3,26.1,34.9,4.0,256.0,NaN,60.0,72.0,NaN,NaN,NaN,62.0,NaN,0.0,NaN,NaN,NaN,0,1.00,0,0,0
4,10006269,27357430.0,2124-06-26,NaN,4.9,1.0,111.0,12.0,2.0,NaN,4.6,128.0,8.4,6.6,27.2,7.7,225.0,NaN,57.0,100.0,0.77,0.4,45.3,96.0,NaN,1.0,19.0,9.0,3.0,0,142.41,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,19994772,29606061.0,2180-12-18,NaN,NaN,0.8,155.0,NaN,NaN,NaN,3.6,137.0,11.6,13.0,36.0,8.5,257.0,NaN,70.0,295.0,NaN,NaN,NaN,0.0,NaN,100.0,NaN,NaN,NaN,0,18600.00,1,0,1
9257,19994772,NaN,2181-03-13,NaN,NaN,0.7,140.0,NaN,NaN,180.0,3.7,140.0,13.0,7.2,36.5,5.6,218.0,NaN,1.0,590.0,NaN,NaN,NaN,1.0,NaN,90.0,NaN,NaN,NaN,0,7800.00,1,0,1
9258,19995080,23057109.0,2147-07-13,NaN,2.3,0.7,53.0,NaN,NaN,NaN,3.8,135.0,10.7,24.3,32.3,6.6,377.0,27.2,42.0,44.0,NaN,NaN,NaN,88.0,NaN,8.0,NaN,NaN,NaN,0,3.00,0,0,0
9259,19997843,NaN,2120-11-18,99.0,3.4,0.8,87.0,8.0,5.0,453.0,3.9,145.0,11.0,18.5,32.3,11.8,88.0,30.6,87.0,21.0,0.83,0.4,53.9,55.0,NaN,9.0,NaN,NaN,NaN,0,0.00,0,0,0


In [204]:
aim1_dataset_label_3to1.to_excel("E:/CNSI_project/demo_and_data_5.0/Processed_data/paper/aim1_dataset_label_3to1.xlsx",index=False)

In [205]:
aim1_dataset_label_3to1['Label_merged'].value_counts()

0    8857
1     404
Name: Label_merged, dtype: int64